<a href="https://colab.research.google.com/github/WilliamJWen/Project42/blob/main/colab_notebooks/visualization_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


Your Model with Dummy Input


In [11]:
##########################################
# Copy Your Model code here #

class Baseline(nn.Module):
  def __init__(self):
    super(Baseline, self).__init__()

    # Hidden layer activation
    self.activation = F.relu

    # Pooling
    self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 2w x 2h -> w x h
    self.avgpool4 = nn.AvgPool2d(kernel_size=4, stride=4) # 4w x 4h -> w x h

    # Convolutional layers
    self.conv1 = nn.Conv2d(in_channels=3,
                           out_channels=64,
                           kernel_size=7,
                           padding=3,
                           stride=2)
    self.conv2 = nn.Conv2d(in_channels=64,
                           out_channels=64,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv3 = nn.Conv2d(in_channels=64,
                           out_channels=64,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv4 = nn.Conv2d(in_channels=64,
                           out_channels=128,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv5 = nn.Conv2d(in_channels=128,
                           out_channels=128,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv6 = nn.Conv2d(in_channels=128,
                           out_channels=256,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv7 = nn.Conv2d(in_channels=256,
                           out_channels=256,
                           kernel_size=3,
                           padding=1,
                           stride=1)

    # FC layer
    self.fc = nn.Linear(256 * 4 * 4, 1)

    # Batch normalization
    self.norm64 = nn.BatchNorm2d(num_features=64)
    self.norm128 = nn.BatchNorm2d(num_features=128)
    self.norm256 = nn.BatchNorm2d(num_features=256)


  def downsample2(self, x, out_channels):
    N, C, H, W = x.shape # Assume H == W

    # Downsample by 2
    downsample = nn.AvgPool2d(2, 2)
    x = downsample(x)

    # 0 padding new channels
    new_channels = out_channels - C
    x = F.pad(x, pad=(0, 0, 0, 0, 0, new_channels))

    return x


  def forward(self, x):
    # Layer 1
    x = self.conv1(x)                                       # Output: 64x128x128
    x = self.norm64(self.activation(self.maxpool2(x)))      # Output: 64x64x64

    # Layer 2
    skip = x.detach().clone()
    x = self.norm64(self.activation(self.conv2(x)))         # Output: 64x64x64

    # Layer 3
    x = self.norm64(self.activation(skip + self.conv3(x)))  # Output: 64x64x64

    # Layer 4
    skip = x.detach().clone()
    x = self.norm128(self.conv4(x))                         # Output: 128x32x32

    # Layer 5
    x = self.norm128(self.activation(self.downsample2(skip, 128)
                                      + self.conv5(x)))     # Output: 128x32x32

    # Layer 6
    skip = x.detach().clone()
    x = self.norm256(self.activation(self.conv6(x)))        # Output: 256x16x16

    # Layer 7
    x = self.downsample2(skip, 256) + self.conv7(x)         # Output: 256x16x16
    x = self.norm256(self.activation(self.avgpool4(x)))     # Output: 256x4x4

    # Layer 8
    x = x.view(-1, 256 * 4 * 4)
    x = self.fc(x)
    x = x.squeeze(1)

    return x


model = Baseline()
num_params = 0
for param in model.parameters():
    num_params += param.numel()
print("There are", num_params, "parameters in the baseline model")

#############################################
# Customize your dummy_input
dummy_input = torch.randn(1, 3, 256, 256)

There are 1195009 parameters in the baseline model


# Netron

In [8]:
!pip install netron

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.3 MB/s eta 0:00:00


In [16]:
import torch.onnx
import netron

In [15]:
!pip install onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 63.2 MB/s eta 0:00:00


In [17]:
torch.onnx.export(model, dummy_input, "model.onnx")

# Open the .onnx file in https://netron.app/

Serving 'model.onnx' at http://localhost:8081


('localhost', 8081)

Serving '/content/model.onnx' at http://localhost:24345

Stopping http://localhost:24345
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/netron/server.py", line 265, in wait
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/netron", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/netron/__init__.py", line 37, in main
    wait()
  File "/usr/local/lib/python3.11/dist-packages/netron/server.py", line 268, in wait
    stop()
  File "/usr/local/lib/python3.11/dist-packages/netron/server.py", line 250, in stop
    thread.stop()
  File "/usr/local/lib/python3.11/dist-packages/netron/server.py", line 158, in stop
    self.stop_event.set()
  File "/usr/lib/python3.11/threading.py", line 599, in set
    self._cond.notify_all()
  File "/usr/lib/python3.11/threading.py", line 403, in noti

# Torchviz

In [3]:
!pip install torchviz
from torchviz import make_dot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
##########################################
# Copy Your Model code here #

class Baseline(nn.Module):
  def __init__(self):
    super(Baseline, self).__init__()

    # Hidden layer activation
    self.activation = F.relu

    # Pooling
    self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 2w x 2h -> w x h
    self.avgpool4 = nn.AvgPool2d(kernel_size=4, stride=4) # 4w x 4h -> w x h

    # Convolutional layers
    self.conv1 = nn.Conv2d(in_channels=3,
                           out_channels=64,
                           kernel_size=7,
                           padding=3,
                           stride=2)
    self.conv2 = nn.Conv2d(in_channels=64,
                           out_channels=64,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv3 = nn.Conv2d(in_channels=64,
                           out_channels=64,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv4 = nn.Conv2d(in_channels=64,
                           out_channels=128,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv5 = nn.Conv2d(in_channels=128,
                           out_channels=128,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv6 = nn.Conv2d(in_channels=128,
                           out_channels=256,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv7 = nn.Conv2d(in_channels=256,
                           out_channels=256,
                           kernel_size=3,
                           padding=1,
                           stride=1)

    # FC layer
    self.fc = nn.Linear(256 * 4 * 4, 1)

    # Batch normalization
    self.norm64 = nn.BatchNorm2d(num_features=64)
    self.norm128 = nn.BatchNorm2d(num_features=128)
    self.norm256 = nn.BatchNorm2d(num_features=256)


  def downsample2(self, x, out_channels):
    N, C, H, W = x.shape # Assume H == W

    # Downsample by 2
    downsample = nn.AvgPool2d(2, 2)
    x = downsample(x)

    # 0 padding new channels
    new_channels = out_channels - C
    x = F.pad(x, pad=(0, 0, 0, 0, 0, new_channels))

    return x


  def forward(self, x):
    # Layer 1
    x = self.conv1(x)                                       # Output: 64x128x128
    x = self.norm64(self.activation(self.maxpool2(x)))      # Output: 64x64x64

    # Layer 2
    skip = x.detach().clone()
    x = self.norm64(self.activation(self.conv2(x)))         # Output: 64x64x64

    # Layer 3
    x = self.norm64(self.activation(skip + self.conv3(x)))  # Output: 64x64x64

    # Layer 4
    skip = x.detach().clone()
    x = self.norm128(self.conv4(x))                         # Output: 128x32x32

    # Layer 5
    x = self.norm128(self.activation(self.downsample2(skip, 128)
                                      + self.conv5(x)))     # Output: 128x32x32

    # Layer 6
    skip = x.detach().clone()
    x = self.norm256(self.activation(self.conv6(x)))        # Output: 256x16x16

    # Layer 7
    x = self.downsample2(skip, 256) + self.conv7(x)         # Output: 256x16x16
    x = self.norm256(self.activation(self.avgpool4(x)))     # Output: 256x4x4

    # Layer 8
    x = x.view(-1, 256 * 4 * 4)
    x = self.fc(x)
    x = x.squeeze(1)

    return x


baseline_model = Baseline()
num_params = 0
for param in baseline_model.parameters():
    num_params += param.numel()
print("There are", num_params, "parameters in the baseline model")


There are 1195009 parameters in the baseline model


In [5]:
model = Baseline()
dummy_input = torch.randn(1, 3, 256, 256)

output = model(dummy_input)
make_dot(output, params=dict(model.named_parameters())).render("model_graph", format="png")

'model_graph.png'

In [20]:
from graphviz import Digraph

In [22]:

dot = Digraph()

dot.node("Input", "28x28x1")
dot.node("Conv1", "Conv (5x5)")
dot.node("Pool1", "Max-Pool (2x2)")
dot.node("Conv2", "Conv (5x5)")
dot.node("Pool2", "Max-Pool (2x2)")
dot.node("Flatten", "Flatten")
dot.node("FC1", "Fully-Connected")
dot.node("FC2", "Fully-Connected (Output)")

# Correctly define edges as a list of (tail, head) tuples
dot.edges([("Input", "Conv1"),  # Edge from Input to Conv1
           ("Conv1", "Pool1"),  # Edge from Conv1 to Pool1
           ("Pool1", "Conv2"),  # Edge from Pool1 to Conv2
           ("Conv2", "Pool2"),  # Edge from Conv2 to Pool2
           ("Pool2", "Flatten"),# Edge from Pool2 to Flatten
           ("Flatten", "FC1"),  # Edge from Flatten to FC1
           ("FC1", "FC2")])    # Edge from FC1 to FC2


dot.render("cnn_architecture", format="png", cleanup=True)


'cnn_architecture.png'